In [1]:
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from tda.experiments.ocsvm_detector.ocsvm_detector_binary import (Config, get_all_embeddings, logger,
                                                                  evaluate_all_embeddings)
from tda.embeddings import EmbeddingType, KernelType, get_gram_matrix
from tda.models.architectures import mnist_lenet

2020-01-15 18:26:21,444 - Devices - INFO - Found 0 compatible with CUDA


In [3]:
n_jobs = 8
config = Config(
    embedding_type=EmbeddingType.PersistentDiagramRipser,
    kernel_type=KernelType.SlicedWasserstein,
    thresholds='0.9',
    epochs=50,
    dataset="MNIST",
    architecture=mnist_lenet.name,
    train_noise=0.0,
    dataset_size=20,
    successful_adv=1,
    attack_type="FGSM",
    identical_train_samples=1,
    noise=0.0,
    
    num_iter=1,
    height=1,
    hash_size=1,
    node_labels=0,
    steps=1,
    
    n_jobs=n_jobs
)

In [ ]:
epsilons = np.linspace(0, 0.4, num=8)
(embedding_train, embedding_test, adv_embeddings, thresholds, stats,
 stats_inf) = get_all_embeddings(config, epsilons=epsilons)

2020-01-15 18:26:32,586 - Datasets - INFO - Instantiated dataset MNIST with validation_size 1000
2020-01-15 18:26:32,587 - Models - INFO - Filename = /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../trained_models/mnist_mnist_lenet_50_epochs.model 

/home/elvis/anaconda3/envs/tda/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'tda.models.architectures.Architecture' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
2020-01-15 18:26:32,605 - Models - INFO - Loaded successfully model from /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../trained_models/mnist_mnist_lenet_50_epochs.model
2020-01-15 18:26:32,606 - Thresholds - INFO - Detected uniform threshold
2020-01-15 18:26:32,606 - Thresholds - INFO - My received thresholds {(-1, 0): 0.9, (0, 1): 0.9,

In [ ]:
logger.info(f"Using kernel {config.kernel_type} with embeddings {config.embedding_type}")

if config.kernel_type == KernelType.RBF:
    param_space = [
        {'gamma': gamma}
        for gamma in np.logspace(-6, -3, 10)
    ]
elif config.kernel_type == KernelType.SlicedWasserstein:
    param_space = [
        {'M': 20, 'sigma': 5 * 10 ** (-1)},
    ]
else:
    raise NotImplementedError(f"Unknown kernel {config.kernel_type}")

t0 = time.time()
gram_train_matrices = {i: get_gram_matrix(
    kernel_type=config.kernel_type,
    embeddings_in=embedding_train,
    embeddings_out=None,
    params=param
)
    for i, param in enumerate(param_space)
}
dt = time.time() - t0
logger.info(f"Computed all Gram train matrices !")
logger.info("Time spent computing gram matrices (train): %s" % dt)

In [ ]:
all_results = evaluate_all_embeddings(
    gram_train_matrices=gram_train_matrices,
    embeddings_train=embedding_train,
    embeddings_test=embedding_test,
    adv_embeddings=adv_embeddings,
    param_space=param_space,
    config=config)

logger.info(all_results)

In [ ]:
accs = list(all_results.values())

In [ ]:
plt.plot(epsilons, accs, linewidth=2)
plt.xlabel("$\epsilon$")
plt.ylabel("detection AUC")
plt.tight_layout()
plt.savefig("attack_detector.png", dpi=200, bbox_inches="tight")